<a href="https://colab.research.google.com/github/Tstrebe2/predicting-text-difficulty/blob/tim-updates/all-models-cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
import sys

# !{sys.executable} -m pip install -U xgboost==1.6.0 -q
!{sys.executable} -m pip install -U pnglatex -q
!{sys.executable} -m pip install -U pdflatex -q

In [29]:
from google.colab import drive
import joblib
import pickle
import os

import pandas as pd
import numpy as np
import scipy.stats as st
from sklearn import metrics

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# If downloading from GitHub, these paths will not work.
repo_path = '/content/drive/MyDrive/milestone-ii/'
train_path = ''.join([repo_path, 'Training_set.csv'])
test_path = ''.join([repo_path, 'Testing_set.csv'])

df_train = pd.read_csv(train_path, sep='\t', index_col='ix')
df_train['lemmatized_text'] = df_train['lemmatized_text'].fillna('')
df_test = pd.read_csv(test_path, sep='\t', index_col='ix')
df_test['lemmatized_text'] = df_test['lemmatized_text'].fillna('')

df = pd.concat([df_train, df_test])
df.head(2)

,original_text,lemmatized_text,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas,label
ix,,,,,,,,,,,
0,There is manuscript evidence that Austen conti...,there be manuscript evidence that austen conti...,8.236551,5.809310,3.57,12.12,2.495517,1.33,4.57,37.0,1
1,"In a remarkable comparative analysis , Mandaea...",in a remarkable comparative analysis mandaean ...,12.320171,7.499286,2.89,11.94,2.251429,1.46,3.77,21.0,1


In [4]:
model_dir = '/content/drive/MyDrive/milestone-ii/Models'
model_map = [
  ('dummyclassifier-clf.joblib', 'Dummy Classifier'),
  ('decisontrees-clf.joblib', 'Decision Tree'),
  ('xgboost-clf.joblib', 'XGBoost RF'),
  ('random-forest-clf.joblib', 'Random Forest'),
  ('logistic-regression-clf.joblib','Logistic Regression'),
  ('multinomialnaivebayes-clf.joblib', 'Naive Bayes'),
  ('svc-model_final.joblib', 'Support Vector Machines'),
]

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# cv_results = {}
# n_splits=10

# skfold = StratifiedKFold(n_splits=n_splits)

# for model_path, model_name in model_map:
#   clf = joblib.load(os.path.join(model_dir, model_path))

#   train_accuracy_scores = np.zeros(n_splits)
#   train_roc_auc_scores = np.zeros(n_splits)
#   train_f1_scores = np.zeros(n_splits)

#   test_accuracy_scores = np.zeros(n_splits)
#   test_roc_auc_scores = np.zeros(n_splits)
#   test_f1_scores = np.zeros(n_splits)

#   for i, (train_ix, test_ix) in enumerate(skfold.split(df, df.label)):
#     X_train, y_train = df.iloc[train_ix, :-1], df.iloc[train_ix].label
#     X_test, y_test = df.iloc[test_ix, :-1], df.iloc[test_ix].label

#     clf.fit(X_train, y_train)

#     y_proba = clf.predict_proba(X_train)[:, 1]
#     y_hat = clf.predict(X_train)
    
#     train_accuracy_scores[i] = metrics.accuracy_score(y_train, y_hat)
#     train_roc_auc_scores[i] = metrics.roc_auc_score(y_train, y_proba)
#     train_f1_scores[i] = metrics.f1_score(y_train, y_hat)

#     y_proba = clf.predict_proba(X_test)[:, 1]
#     y_hat = clf.predict(X_test)

#     test_accuracy_scores[i] = metrics.accuracy_score(y_test, y_hat)
#     test_roc_auc_scores[i] = metrics.roc_auc_score(y_test, y_proba)
#     test_f1_scores[i] = metrics.f1_score(y_test, y_hat)
#     print(model_name, f'fit {i+1} complete')

#   cv_results[model_name] = dict(
#       train_accuracy_scores=train_accuracy_scores,
#       train_roc_auc_scores=train_roc_auc_scores,
#       train_f1_scores=train_f1_scores,

#       test_accuracy_scores=test_accuracy_scores,
#       test_roc_auc_scores=test_roc_auc_scores,
#       test_f1_scores=test_f1_scores,
#   )
#   print(cv_results[model_name])

In [10]:
# with open('/content/drive/MyDrive/milestone-ii/all-model-cv-results', 'wb') as fh:
#   pickle.dump(cv_results, fh)

In [2]:
with open('/content/drive/MyDrive/milestone-ii/all-model-cv-results', 'rb') as fh:
  cv_results = pickle.load(fh)
  print(cv_results.keys())

dict_keys(['Dummy Classifier', 'Decision Tree', 'XGBoost RF', 'Random Forest', 'Naive Bayes', 'Logistic Regression', 'Support Vector Machines'])


In [10]:
cv_results['Dummy Classifier']

{'train_accuracy_scores': array([0.49929217, 0.5012277 , 0.50118505, 0.50048122, 0.499996  ,
        0.50044389, 0.5005452 , 0.49962009, 0.50049055, 0.49960276]),
 'train_roc_auc_scores': array([0.49936949, 0.49930284, 0.49915354, 0.49886827, 0.5004279 ,
        0.5006705 , 0.5001613 , 0.50008398, 0.50130901, 0.49940548]),
 'train_f1_scores': array([0.50005723, 0.500927  , 0.5005779 , 0.50052649, 0.50044083,
        0.50129215, 0.50056384, 0.49959741, 0.50043727, 0.49929975]),
 'test_accuracy_scores': array([0.500012  , 0.50224344, 0.50008398, 0.49818845, 0.50205149,
        0.49948413, 0.50034791, 0.49773256, 0.50064785, 0.50019196]),
 'test_roc_auc_scores': array([0.50142761, 0.500276  , 0.49886032, 0.50406701, 0.50401907,
        0.49636494, 0.49950811, 0.50274718, 0.49628083, 0.50424705]),
 'test_f1_scores': array([0.5013401 , 0.50229121, 0.50139996, 0.49776668, 0.50049342,
        0.49809923, 0.49963958, 0.49628221, 0.50094722, 0.50143609])}

In [138]:
columns = ['Model', 'Mean ROC AUC', 'ROC AUC (95 % CI)', 'Mean Accuracy', 'Accuracy (95 % CI)', 'Mean F1', 'F1 (95 % CI)']
results = pd.DataFrame(columns=columns)

for _, model_name in model_map:
  y_auc = cv_results[model_name]['test_roc_auc_scores']
  y_acc = cv_results[model_name]['test_accuracy_scores']
  y_f1 = cv_results[model_name]['test_f1_scores']

  mean_auc = np.round(y_auc.mean(), 4)
  mean_acc = np.round(y_acc.mean(), 4)
  mean_f1  = np.round(y_f1.mean(),  4)

  low_auc, high_auc = st.t.interval(alpha=0.95, df=y_auc.shape[0]-1, loc=mean_auc, scale=st.sem(y_auc))
  low_auc, high_auc = np.round(low_auc, 4), np.round(high_auc, 4)
  low_acc, high_acc = st.t.interval(alpha=0.95, df=y_acc.shape[0]-1, loc=mean_acc, scale=st.sem(y_acc))
  low_acc, high_acc = np.round(low_acc, 4), np.round(high_acc, 4)
  low_f1, high_f1 = st.t.interval(alpha=0.95, df=y_f1.shape[0]-1, loc=mean_f1, scale=st.sem(y_f1))
  low_f1, high_f1 = np.round(low_f1, 4), np.round(high_f1, 4)

  results = results.append(pd.DataFrame([[model_name, 
                                          mean_auc, str((low_auc, high_auc)), 
                                          mean_acc, str((low_acc, high_acc)),
                                          mean_f1, str((low_f1, high_f1))]], columns=columns), ignore_index=True)

print(results.to_latex(index=False, column_format=format))

\begin{tabular}{l@{\hskip 12pt}S[table-format = 2.2]S[table-format = 2.2]S[table-format = 2.2]S[table-format = 2.2]}
\toprule
                  Model &  Mean ROC AUC & ROC AUC (95 \% CI) &  Mean Accuracy & Accuracy (95 \% CI) &  Mean F1 &     F1 (95 \% CI) \\
\midrule
       Dummy Classifier &        0.5008 &   (0.4986, 0.503) &         0.5001 &   (0.4991, 0.5011) &   0.5000 & (0.4986, 0.5014) \\
          Decision Tree &        0.7216 &  (0.7205, 0.7227) &         0.6586 &   (0.6577, 0.6595) &   0.6842 & (0.6829, 0.6855) \\
             XGBoost RF &        0.7473 &  (0.7464, 0.7482) &         0.6757 &   (0.6745, 0.6769) &   0.6856 &  (0.684, 0.6872) \\
          Random Forest &        0.7500 &  (0.7489, 0.7511) &         0.6787 &   (0.6777, 0.6797) &   0.6939 & (0.6927, 0.6951) \\
    Logistic Regression &        0.7717 &  (0.7706, 0.7728) &         0.7005 &   (0.6994, 0.7016) &   0.7038 & (0.7024, 0.7052) \\
            Naive Bayes &        0.7846 &  (0.7797, 0.7895) &         0.7076